## ******l'essai sur nos données******

In [16]:
import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import butter, filtfilt, detrend, periodogram
from neural_methods.model.DeepPhys import DeepPhys
import random


******séance1******

In [13]:

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
cv2.setNumThreads(0)
cv2.ocl.setUseOpenCL(False)


video_folder = "C:/Users/BIG BOX/Downloads/dat/Séance1/subject01"  
fps = 20.45
save_csv = True  
output_folder = "results/rppg_signals_csv"  

os.makedirs(output_folder, exist_ok=True)

def butter_bandpass_filter(data, lowcut=0.83, highcut=3.33, fs=20.0, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

def preprocess_rppg(signal, fs):
    signal = detrend(signal)
    return butter_bandpass_filter(signal, fs=fs)

def compute_bpm(signal, fs):
    if len(signal) < 10:
        return np.nan
    filtered = preprocess_rppg(signal, fs)
    freqs, power = periodogram(filtered, fs=fs)
    mask = (freqs >= 0.83) & (freqs <= 3.33)
    freqs, power = freqs[mask], power[mask]
    if len(freqs) == 0:
        return np.nan
    return freqs[np.argmax(power)] * 60   

def stable_bpm(signal, fs, segment_sec=10):
    segment_len = int(fs * segment_sec)
    bpms = []
    for i in range(0, len(signal) - segment_len, segment_len // 2):
        seg = signal[i:i + segment_len]
        if len(seg) < segment_len:
            continue
        bpm = compute_bpm(seg, fs)
        if not np.isnan(bpm) and 40 <= bpm <= 150:  
            bpms.append(bpm)
    if len(bpms) == 0:
        return np.nan
    return np.median(bpms)  


model = DeepPhys()
model.eval()

def estimate_rppg_from_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (36, 36))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)
    cap.release()

    if len(frames) < 2:
        return None, None

    frames = np.array(frames)
    frames = torch.tensor(frames).permute(0, 3, 1, 2).float() / 255.0
    diffs = frames[1:] - frames[:-1]
    frames = frames[:-1]
    input_tensor = torch.cat([diffs, frames], dim=1)

    results = []
    with torch.no_grad():
        for i in range(input_tensor.shape[0]):
            img = input_tensor[i].unsqueeze(0)
            output = model(img)
            results.append(output.item())
                 
    return np.array(results), fps

resultats = []

for file in sorted(os.listdir(video_folder)):
    if not file.endswith(".mp4"):
        continue

    video_path = os.path.join(video_folder, file)
    print(f" Traitement : {file}")

    signal, fs_used = estimate_rppg_from_video(video_path)
    if signal is None:
        print(f" Impossible de traiter {file}")
        continue

    bpm = bpm = stable_bpm(signal, fs_used)
    
    print(f" BPM estimé pour {file} : {bpm:.2f} BPM")

    if save_csv:
        df = pd.DataFrame({'pred_rPPG': signal})
        name_no_ext = os.path.splitext(file)[0]
        csv_path = os.path.join(output_folder, f"{name_no_ext}.csv")
        df.to_csv(csv_path, index=False)
        print(f" Signal rPPG enregistré dans : {csv_path}")

    resultats.append({'video': file, 'bpm_estime': bpm})

df_resultats = pd.DataFrame(resultats)
print("\n Résumé des fréquences cardiaques estimées :")
display(df_resultats)


 Traitement : vid1.mp4
 BPM estimé pour vid1.mp4 : 132.32 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid1.csv
 Traitement : vid2.mp4
 BPM estimé pour vid2.mp4 : 66.16 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid2.csv
 Traitement : vid3.mp4
 BPM estimé pour vid3.mp4 : 69.17 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid3.csv
 Traitement : vid4.mp4
 BPM estimé pour vid4.mp4 : 72.18 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid4.csv

 Résumé des fréquences cardiaques estimées :


,video,bpm_estime
0,vid1.mp4,132.323529
1,vid2.mp4,66.161765
2,vid3.mp4,69.169118
3,vid4.mp4,72.176471


******séance2******

In [12]:

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
cv2.setNumThreads(0)
cv2.ocl.setUseOpenCL(False)


video_folder = "C:/Users/BIG BOX/Downloads/dat/Séance2/subject02"  
fps = 20.45
save_csv = True  
output_folder = "results/rppg_signals_csv"  

os.makedirs(output_folder, exist_ok=True)

def butter_bandpass_filter(data, lowcut=0.83, highcut=3.33, fs=20.0, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

def preprocess_rppg(signal, fs):
    signal = detrend(signal)
    return butter_bandpass_filter(signal, fs=fs)

def compute_bpm(signal, fs):
    if len(signal) < 10:
        return np.nan
    filtered = preprocess_rppg(signal, fs)
    freqs, power = periodogram(filtered, fs=fs)
    mask = (freqs >= 0.83) & (freqs <= 3.33)
    freqs, power = freqs[mask], power[mask]
    if len(freqs) == 0:
        return np.nan
    return freqs[np.argmax(power)] * 60   

def stable_bpm(signal, fs, segment_sec=10):
    segment_len = int(fs * segment_sec)
    bpms = []
    for i in range(0, len(signal) - segment_len, segment_len // 2):
        seg = signal[i:i + segment_len]
        if len(seg) < segment_len:
            continue
        bpm = compute_bpm(seg, fs)
        if not np.isnan(bpm) and 40 <= bpm <= 150:  
            bpms.append(bpm)
    if len(bpms) == 0:
        return np.nan
    return np.median(bpms)  

model = DeepPhys()
model.eval()

def estimate_rppg_from_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (36, 36))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)
    cap.release()

    if len(frames) < 2:
        return None, None

    frames = np.array(frames)
    frames = torch.tensor(frames).permute(0, 3, 1, 2).float() / 255.0
    diffs = frames[1:] - frames[:-1]
    frames = frames[:-1]
    input_tensor = torch.cat([diffs, frames], dim=1)

    results = []
    with torch.no_grad():
        for i in range(input_tensor.shape[0]):
            img = input_tensor[i].unsqueeze(0)
            output = model(img)
            results.append(output.item())
                 
    return np.array(results), fps

resultats = []

for file in sorted(os.listdir(video_folder)):
    if not file.endswith(".mp4"):
        continue

    video_path = os.path.join(video_folder, file)
    print(f" Traitement : {file}")

    signal, fs_used = estimate_rppg_from_video(video_path)
    if signal is None:
        print(f" Impossible de traiter {file}")
        continue

    bpm = bpm = stable_bpm(signal, fs_used)
    
    print(f" BPM estimé pour {file} : {bpm:.2f} BPM")

    if save_csv:
        df = pd.DataFrame({'pred_rPPG': signal})
        name_no_ext = os.path.splitext(file)[0]
        csv_path = os.path.join(output_folder, f"{name_no_ext}.csv")
        df.to_csv(csv_path, index=False)
        print(f" Signal rPPG enregistré dans : {csv_path}")

    resultats.append({'video': file, 'bpm_estime': bpm})

df_resultats = pd.DataFrame(resultats)
print("\n Résumé des fréquences cardiaques estimées :")
display(df_resultats)


 Traitement : vid1.mp4
 BPM estimé pour vid1.mp4 : 87.21 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid1.csv
 Traitement : vid2.mp4
 BPM estimé pour vid2.mp4 : 84.21 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid2.csv
 Traitement : vid3.mp4
 BPM estimé pour vid3.mp4 : 120.29 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid3.csv
 Traitement : vid4.mp4
 BPM estimé pour vid4.mp4 : 72.18 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid4.csv
 Traitement : vid5.mp4
 BPM estimé pour vid5.mp4 : 93.23 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid5.csv

 Résumé des fréquences cardiaques estimées :


,video,bpm_estime
0,vid1.mp4,87.213235
1,vid2.mp4,84.205882
2,vid3.mp4,120.294118
3,vid4.mp4,72.176471
4,vid5.mp4,93.227941


******séance3******

In [11]:

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
cv2.setNumThreads(0)
cv2.ocl.setUseOpenCL(False)


video_folder = "C:/Users/BIG BOX/Downloads/dat/Séance3/subject03"  
fps = 20.45
save_csv = True  
output_folder = "results/rppg_signals_csv"  

os.makedirs(output_folder, exist_ok=True)

def butter_bandpass_filter(data, lowcut=0.83, highcut=3.33, fs=20.0, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

def preprocess_rppg(signal, fs):
    signal = detrend(signal)
    return butter_bandpass_filter(signal, fs=fs)

def compute_bpm(signal, fs):
    if len(signal) < 10:
        return np.nan
    filtered = preprocess_rppg(signal, fs)
    freqs, power = periodogram(filtered, fs=fs)
    mask = (freqs >= 0.83) & (freqs <= 3.33)
    freqs, power = freqs[mask], power[mask]
    if len(freqs) == 0:
        return np.nan
    return freqs[np.argmax(power)] * 60   

def stable_bpm(signal, fs, segment_sec=10):
    segment_len = int(fs * segment_sec)
    bpms = []
    for i in range(0, len(signal) - segment_len, segment_len // 2):
        seg = signal[i:i + segment_len]
        if len(seg) < segment_len:
            continue
        bpm = compute_bpm(seg, fs)
        if not np.isnan(bpm) and 40 <= bpm <= 150: 
            bpms.append(bpm)
    if len(bpms) == 0:
        return np.nan
    return np.median(bpms)  


model = DeepPhys()
model.eval()

def estimate_rppg_from_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (36, 36))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)
    cap.release()

    if len(frames) < 2:
        return None, None

    frames = np.array(frames)
    frames = torch.tensor(frames).permute(0, 3, 1, 2).float() / 255.0
    diffs = frames[1:] - frames[:-1]
    frames = frames[:-1]
    input_tensor = torch.cat([diffs, frames], dim=1)

    results = []
    with torch.no_grad():
        for i in range(input_tensor.shape[0]):
            img = input_tensor[i].unsqueeze(0)
            output = model(img)
            results.append(output.item())
                 
    return np.array(results), fps

resultats = []

for file in sorted(os.listdir(video_folder)):
    if not file.endswith(".mp4"):
        continue

    video_path = os.path.join(video_folder, file)
    print(f" Traitement : {file}")

    signal, fs_used = estimate_rppg_from_video(video_path)
    if signal is None:
        print(f" Impossible de traiter {file}")
        continue

    bpm = bpm = stable_bpm(signal, fs_used)
    
    print(f" BPM estimé pour {file} : {bpm:.2f} BPM")

    if save_csv:
        df = pd.DataFrame({'pred_rPPG': signal})
        name_no_ext = os.path.splitext(file)[0]
        csv_path = os.path.join(output_folder, f"{name_no_ext}.csv")
        df.to_csv(csv_path, index=False)
        print(f" Signal rPPG enregistré dans : {csv_path}")

    resultats.append({'video': file, 'bpm_estime': bpm})

df_resultats = pd.DataFrame(resultats)
print("\n Résumé des fréquences cardiaques estimées :")
display(df_resultats)


 Traitement : vid1.mp4
 BPM estimé pour vid1.mp4 : 69.17 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid1.csv
 Traitement : vid2.mp4
 BPM estimé pour vid2.mp4 : 60.15 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid2.csv
 Traitement : vid3.mp4
 BPM estimé pour vid3.mp4 : 93.23 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid3.csv
 Traitement : vid4.mp4
 BPM estimé pour vid4.mp4 : 72.18 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid4.csv
 Traitement : vid5.mp4
 BPM estimé pour vid5.mp4 : 102.25 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid5.csv

 Résumé des fréquences cardiaques estimées :


,video,bpm_estime
0,vid1.mp4,69.169118
1,vid2.mp4,60.147059
2,vid3.mp4,93.227941
3,vid4.mp4,72.176471
4,vid5.mp4,102.250000


******séance4******

In [10]:

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
cv2.setNumThreads(0)
cv2.ocl.setUseOpenCL(False)


video_folder = "C:/Users/BIG BOX/Downloads/dat/Séance4/subject04"  
fps = 20.45
save_csv = True  
output_folder = "results/rppg_signals_csv"  

os.makedirs(output_folder, exist_ok=True)

def butter_bandpass_filter(data, lowcut=0.83, highcut=3.33, fs=20.0, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

def preprocess_rppg(signal, fs):
    signal = detrend(signal)
    return butter_bandpass_filter(signal, fs=fs)

def compute_bpm(signal, fs):
    if len(signal) < 10:
        return np.nan
    filtered = preprocess_rppg(signal, fs)
    freqs, power = periodogram(filtered, fs=fs)
    mask = (freqs >= 0.83) & (freqs <= 3.33)
    freqs, power = freqs[mask], power[mask]
    if len(freqs) == 0:
        return np.nan
    return freqs[np.argmax(power)] * 60   

def stable_bpm(signal, fs, segment_sec=10):
    segment_len = int(fs * segment_sec)
    bpms = []
    for i in range(0, len(signal) - segment_len, segment_len // 2):
        seg = signal[i:i + segment_len]
        if len(seg) < segment_len:
            continue
        bpm = compute_bpm(seg, fs)
        if not np.isnan(bpm) and 40 <= bpm <= 150:  
            bpms.append(bpm)
    if len(bpms) == 0:
        return np.nan
    return np.median(bpms)  


model = DeepPhys()
model.eval()

def estimate_rppg_from_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (36, 36))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)
    cap.release()

    if len(frames) < 2:
        return None, None

    frames = np.array(frames)
    frames = torch.tensor(frames).permute(0, 3, 1, 2).float() / 255.0
    diffs = frames[1:] - frames[:-1]
    frames = frames[:-1]
    input_tensor = torch.cat([diffs, frames], dim=1)

    results = []
    with torch.no_grad():
        for i in range(input_tensor.shape[0]):
            img = input_tensor[i].unsqueeze(0)
            output = model(img)
            results.append(output.item())
                 
    return np.array(results), fps

resultats = []

for file in sorted(os.listdir(video_folder)):
    if not file.endswith(".mp4"):
        continue

    video_path = os.path.join(video_folder, file)
    print(f" Traitement : {file}")

    signal, fs_used = estimate_rppg_from_video(video_path)
    if signal is None:
        print(f" Impossible de traiter {file}")
        continue

    bpm = bpm = stable_bpm(signal, fs_used)
    
    print(f" BPM estimé pour {file} : {bpm:.2f} BPM")

    if save_csv:
        df = pd.DataFrame({'pred_rPPG': signal})
        name_no_ext = os.path.splitext(file)[0]
        csv_path = os.path.join(output_folder, f"{name_no_ext}.csv")
        df.to_csv(csv_path, index=False)
        print(f" Signal rPPG enregistré dans : {csv_path}")

    resultats.append({'video': file, 'bpm_estime': bpm})

df_resultats = pd.DataFrame(resultats)
print("\n Résumé des fréquences cardiaques estimées :")
display(df_resultats)


 Traitement : vid1.mp4
 BPM estimé pour vid1.mp4 : 102.25 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid1.csv
 Traitement : vid2.mp4
 BPM estimé pour vid2.mp4 : 126.31 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid2.csv
 Traitement : vid3.mp4
 BPM estimé pour vid3.mp4 : 108.26 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid3.csv
 Traitement : vid4.mp4
 BPM estimé pour vid4.mp4 : 108.26 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid4.csv
 Traitement : vid5.mp4
 BPM estimé pour vid5.mp4 : 66.16 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid5.csv

 Résumé des fréquences cardiaques estimées :


,video,bpm_estime
0,vid1.mp4,102.250000
1,vid2.mp4,126.308824
2,vid3.mp4,108.264706
3,vid4.mp4,108.264706
4,vid5.mp4,66.161765


******séance5******

In [9]:

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
cv2.setNumThreads(0)
cv2.ocl.setUseOpenCL(False)


video_folder = "C:/Users/BIG BOX/Downloads/dat/Séance5/subject05"  
fps = 20.45
save_csv = True  
output_folder = "results/rppg_signals_csv"  

os.makedirs(output_folder, exist_ok=True)

def butter_bandpass_filter(data, lowcut=0.83, highcut=3.33, fs=20.0, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

def preprocess_rppg(signal, fs):
    signal = detrend(signal)
    return butter_bandpass_filter(signal, fs=fs)

def compute_bpm(signal, fs):
    if len(signal) < 10:
        return np.nan
    filtered = preprocess_rppg(signal, fs)
    freqs, power = periodogram(filtered, fs=fs)
    mask = (freqs >= 0.83) & (freqs <= 3.33)
    freqs, power = freqs[mask], power[mask]
    if len(freqs) == 0:
        return np.nan
    return freqs[np.argmax(power)] * 60   

def stable_bpm(signal, fs, segment_sec=10):
    segment_len = int(fs * segment_sec)
    bpms = []
    for i in range(0, len(signal) - segment_len, segment_len // 2):
        seg = signal[i:i + segment_len]
        if len(seg) < segment_len:
            continue
        bpm = compute_bpm(seg, fs)
        if not np.isnan(bpm) and 40 <= bpm <= 150:  
            bpms.append(bpm)
    if len(bpms) == 0:
        return np.nan
    return np.median(bpms)  


model = DeepPhys()
model.eval()

def estimate_rppg_from_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (36, 36))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)
    cap.release()

    if len(frames) < 2:
        return None, None

    frames = np.array(frames)
    frames = torch.tensor(frames).permute(0, 3, 1, 2).float() / 255.0
    diffs = frames[1:] - frames[:-1]
    frames = frames[:-1]
    input_tensor = torch.cat([diffs, frames], dim=1)

    results = []
    with torch.no_grad():
        for i in range(input_tensor.shape[0]):
            img = input_tensor[i].unsqueeze(0)
            output = model(img)
            results.append(output.item())
                 
    return np.array(results), fps

resultats = []

for file in sorted(os.listdir(video_folder)):
    if not file.endswith(".mp4"):
        continue

    video_path = os.path.join(video_folder, file)
    print(f" Traitement : {file}")

    signal, fs_used = estimate_rppg_from_video(video_path)
    if signal is None:
        print(f" Impossible de traiter {file}")
        continue

    bpm = bpm = stable_bpm(signal, fs_used)
    
    print(f" BPM estimé pour {file} : {bpm:.2f} BPM")

    if save_csv:
        df = pd.DataFrame({'pred_rPPG': signal})
        name_no_ext = os.path.splitext(file)[0]
        csv_path = os.path.join(output_folder, f"{name_no_ext}.csv")
        df.to_csv(csv_path, index=False)
        print(f" Signal rPPG enregistré dans : {csv_path}")

    resultats.append({'video': file, 'bpm_estime': bpm})

df_resultats = pd.DataFrame(resultats)
print("\n Résumé des fréquences cardiaques estimées :")
display(df_resultats)


 Traitement : vid1.mp4
 BPM estimé pour vid1.mp4 : 114.28 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid1.csv
 Traitement : vid2.mp4
 BPM estimé pour vid2.mp4 : 102.25 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid2.csv
 Traitement : vid3.mp4
 BPM estimé pour vid3.mp4 : 72.18 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid3.csv
 Traitement : vid4.mp4
 BPM estimé pour vid4.mp4 : 117.29 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid4.csv
 Traitement : vid5.mp4
 BPM estimé pour vid5.mp4 : 114.28 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid5.csv

 Résumé des fréquences cardiaques estimées :


,video,bpm_estime
0,vid1.mp4,114.279412
1,vid2.mp4,102.250000
2,vid3.mp4,72.176471
3,vid4.mp4,117.286765
4,vid5.mp4,114.279412


******séance6******

In [17]:

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
cv2.setNumThreads(0)
cv2.ocl.setUseOpenCL(False)


video_folder = "C:/Users/BIG BOX/Downloads/dat/Séance6/subject06"  
fps = 20.45
save_csv = True  
output_folder = "results/rppg_signals_csv"  

os.makedirs(output_folder, exist_ok=True)

def butter_bandpass_filter(data, lowcut=0.83, highcut=3.33, fs=20.0, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

def preprocess_rppg(signal, fs):
    signal = detrend(signal)
    return butter_bandpass_filter(signal, fs=fs)

def compute_bpm(signal, fs):
    if len(signal) < 10:
        return np.nan
    filtered = preprocess_rppg(signal, fs)
    freqs, power = periodogram(filtered, fs=fs)
    mask = (freqs >= 0.83) & (freqs <= 3.33)
    freqs, power = freqs[mask], power[mask]
    if len(freqs) == 0:
        return np.nan
    return freqs[np.argmax(power)] * 60   

def stable_bpm(signal, fs, segment_sec=10):
    segment_len = int(fs * segment_sec)
    bpms = []
    for i in range(0, len(signal) - segment_len, segment_len // 2):
        seg = signal[i:i + segment_len]
        if len(seg) < segment_len:
            continue
        bpm = compute_bpm(seg, fs)
        if not np.isnan(bpm) and 40 <= bpm <= 150:  
            bpms.append(bpm)
    if len(bpms) == 0:
        return np.nan
    return np.median(bpms)  


model = DeepPhys()
model.eval()

def estimate_rppg_from_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (36, 36))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)
    cap.release()

    if len(frames) < 2:
        return None, None

    frames = np.array(frames)
    frames = torch.tensor(frames).permute(0, 3, 1, 2).float() / 255.0
    diffs = frames[1:] - frames[:-1]
    frames = frames[:-1]
    input_tensor = torch.cat([diffs, frames], dim=1)

    results = []
    with torch.no_grad():
        for i in range(input_tensor.shape[0]):
            img = input_tensor[i].unsqueeze(0)
            output = model(img)
            results.append(output.item())
                 
    return np.array(results), fps

resultats = []

for file in sorted(os.listdir(video_folder)):
    if not file.endswith(".mp4"):
        continue

    video_path = os.path.join(video_folder, file)
    print(f" Traitement : {file}")

    signal, fs_used = estimate_rppg_from_video(video_path)
    if signal is None:
        print(f" Impossible de traiter {file}")
        continue

    bpm = bpm = stable_bpm(signal, fs_used)
    
    print(f" BPM estimé pour {file} : {bpm:.2f} BPM")

    if save_csv:
        df = pd.DataFrame({'pred_rPPG': signal})
        name_no_ext = os.path.splitext(file)[0]
        csv_path = os.path.join(output_folder, f"{name_no_ext}.csv")
        df.to_csv(csv_path, index=False)
        print(f" Signal rPPG enregistré dans : {csv_path}")

    resultats.append({'video': file, 'bpm_estime': bpm})

df_resultats = pd.DataFrame(resultats)
print("\n Résumé des fréquences cardiaques estimées :")
display(df_resultats)


 Traitement : vid1.mp4
 BPM estimé pour vid1.mp4 : 120.29 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid1.csv
 Traitement : vid2.mp4
 BPM estimé pour vid2.mp4 : 117.29 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid2.csv
 Traitement : vid3.mp4
 BPM estimé pour vid3.mp4 : 84.21 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid3.csv
 Traitement : vid4.mp4
 BPM estimé pour vid4.mp4 : 90.22 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid4.csv
 Traitement : vid5.mp4
 BPM estimé pour vid5.mp4 : 138.34 BPM
 Signal rPPG enregistré dans : results/rppg_signals_csv\vid5.csv

 Résumé des fréquences cardiaques estimées :


,video,bpm_estime
0,vid1.mp4,120.294118
1,vid2.mp4,117.286765
2,vid3.mp4,84.205882
3,vid4.mp4,90.220588
4,vid5.mp4,138.338235
